In [28]:
import data
import utils
import json
import torch
import torchvision
import numpy as np
import sklearn.metrics as skm
import torchvision.models.segmentation.deeplabv3 as dlv3
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from tqdm import tqdm

## Test-related functions

In [29]:
def eval(model, test_dataloader, tgt_num_classes, tgt_ignore_label=None):
    model.eval()
    confusion_mat = np.zeros(shape=(tgt_num_classes, tgt_num_classes), dtype=np.int64)
    
    with torch.no_grad():
        for (inputs, targets) in tqdm(test_dataloader, desc='Batch progress'):
            ipts = inputs.to('cuda', non_blocking=True)# torch.autograd.Variable(inputs).cuda()
            tgts = targets.to('cuda', non_blocking=True)# torch.autograd.Variable(targets).cuda()
            
            preds = model(ipts)['out']
            preds = torch.argmax(preds.cpu(), dim=1)
            tgts = torch.squeeze(targets, dim=1)

            confusion_mat = np.add(confusion_mat, utils.compute_confusion_matrix(predictions=preds, targets=tgts, num_classes=tgt_num_classes))
        
        miou, ious = utils.measure_performance(confusion_mat=confusion_mat,
                                               num_classes=tgt_num_classes,
                                               ignore_label=tgt_ignore_label)

    return {'miou': np.round(miou, 4), 'ious': np.round(ious, 4)}

## Test configuration loading

In [30]:
import pprint
args = {}
with open('G:/My Drive/Master IVA/Master Thesis/Models/20230410_052707/eval-config.json') as json_file:
    args = json.load(json_file)
pprint.pprint(args)


{'data_source': 'real',
 'ignore_label': 19,
 'image_height': 256,
 'image_width': 512,
 'model_load_folder_path': 'G:/My Drive/Master IVA/Master '
                           'Thesis/Models/20230409_211303',
 'test_batch_size': 4,
 'test_subset_size': 200,
 'train_mean': [0.39532706141471863, 0.3791861832141876, 0.36232122778892517],
 'train_std': [0.20954108238220215, 0.20490853488445282, 0.20333392918109894]}


## Data loading and preprocessing

In [31]:
test_dataset = data.HybridDataset(
        root_path=f'E:/Datasets/{args["data_source"]}/val',
        input_dir='rgb',
        target_dir='semantic_segmentation_mapped',
        ipt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"])),
                                                      torchvision.transforms.Normalize(mean=args["train_mean"], std=args["train_std"])]),
        tgt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),                        
        type=args['data_source'],
        labels_mapping=None)
test_dataset = Subset(test_dataset, indices=range(args['test_subset_size']))
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=args['test_batch_size'],
    shuffle=False
)

## Model under test preparation

In [32]:
# File parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = dlv3.deeplabv3_resnet50(weights=None, progress=True, num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()))
model.load_state_dict(torch.load(f'{args["model_load_folder_path"]}/deeplabv3_model.pt'))
model.to(device)
print('Loaded model to device.')

Loaded model to device.


In [33]:
test_metrics = eval(model=model, 
                    test_dataloader=test_dataloader, 
                    tgt_num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()),
                    tgt_ignore_label=None)
test_metrics['ious'] = {k:v for k,v in zip(data.SemanticLabelMapper.ID_TO_STRING['common'].values(),
                                                   test_metrics['ious'])}


Batch progress: 100%|██████████| 50/50 [00:34<00:00,  1.46it/s]


In [34]:
print(test_metrics)
test_metrics = json.dumps(test_metrics, indent=4)
with open(f'{args["model_load_folder_path"]}/eval_metrics.json', mode='w') as f:
    f.write(test_metrics)

{'miou': 0.1222, 'ious': {'road': 0.2223, 'sidewalk': 0.0857, 'building': 0.5319, 'wall': 0.0262, 'fence': 0.001, 'trafficlight': 0.0149, 'trafficsign': 0.0256, 'vegetation': 0.6436, 'terrain': 0.0437, 'pedestrian': 0.114, 'rider': 0.0222, 'car': 0.1486, 'truck': 0.0066, 'bus': 0.0227, 'motorcycle': 0.0018, 'bicycle': 0.0, 'background': 0.1663}}
